# More Theories and Models (Continue)

## Exponential Smoothing (ETS)

## Holt Winter’s Exponential Smoothing

## ARIMA

## Meta-Learning

## Reference

TODO: More on the R Forecast Liberary

# Using Deep Learning to Forecast

## Using Keras to Build Recursive Nerual Networks (RNN)

## Recent development on DL

# Summary

## Comparison of Various Algorithms